# Extracción de características

La extracción de características consiste en extraer toda la información posible y relevante de las reviews de cada estación. Esto nos permitirá dar recomendaciones de las rutas más personalizadas al poder aplicar filtros con la información que consigamos obtener a través de las reviews.

## Imports necesarios

In [52]:
import nltk
from nltk.tokenize import word_tokenize 
nltk.download('punkt_tab')
from nltk.corpus import stopwords
nltk.download("stopwords")
import spacy
from collections import defaultdict, Counter
import stanza
stanza.download("es")  # Descargar el modelo en español
from deep_translator import GoogleTranslator
from nltk.corpus import wordnet as wn
from unidecode import unidecode

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\evano\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\evano\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2025-04-27 13:55:36 INFO: Downloaded file to C:\Users\evano\stanza_resources\resources.json
2025-04-27 13:55:36 INFO: Downloading default packages for language: es (Spanish) ...
2025-04-27 13:55:38 INFO: File exists: C:\Users\evano\stanza_resources\es\default.zip
2025-04-27 13:55:44 INFO: Finished downloading models and saved to C:\Users\evano\stanza_resources


## Implementación de métodos para extracción de características

### N-gramas

Para la extracción de características vamos a emplear n-gramas. Un n-grama es una secuencia de n palabras consecutivas en un texto. Esto nos va a permitir poder sacar la infomación relevante de las reseñas al poder aplicar distintos n-gramas y ver sus relaciones.

Vamos primero a probar con unas reseñas de prueba para ver como de bien funcionan los n-gramas:

In [2]:
reviews = [
    "La estación de metro es moderna y limpia, pero los trenes son lentos.",
    "A veces hay demasiada gente y el servicio es malo.",
    "El metro de Madrid es rápido y eficiente, aunque algunas estaciones están sucias.",
    "Buena conexión con otras líneas, pero los horarios nunca son confiables."
]

Para asegurarnos de conseguir la información más relevante vamos a eliminar de nuestras reviews de prueba todas las stopwords

Eliminamos las stopwords en español de las reviews de prueba

In [3]:
#Obtenemos las stopwords en español
spanishStopwords = list(stopwords.words("spanish"))

# Eliminar stopwords de cada review
filteredReviews = [
    " ".join([word for word in word_tokenize(review.lower()) if word not in spanishStopwords and word.isalnum()])
    for review in reviews
]

# Imprimir resultado
print(filteredReviews)

['estación metro moderna limpia trenes lentos', 'veces demasiada gente servicio malo', 'metro madrid rápido eficiente aunque estaciones sucias', 'buena conexión líneas horarios nunca confiables']


Una vez eliminadas las stopwords procedemos a usar los n-gramas para poder analizarlas y extraer las características. 
Para que sea lo más fiable posible vamos a usar n-gramas de 2 a 5, es decir, desde bigramas hasta pentagramas, para obtener una precisión que sea la mejor posible. Si solo tenemos bigramas nos vamos a dejar información relevante pero si usamos n-gramas muy grandes la precisión también se vería reducida, por lo que se optó por usar n-gramas de 2 a 5.

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

# Crear un vectorizador de ngramas
vectorizer = CountVectorizer(ngram_range=(2, 5))

# Aplicar a las reseñas
X = vectorizer.fit_transform(filteredReviews)

# Ver n-gramas encontrados
ngramas = vectorizer.get_feature_names_out()
print("ngramas encontrados:", ngramas)

# Ver la matriz de frecuencia
print("Matriz de ngramas:\n", X.toarray())

ngramas encontrados: ['aunque estaciones' 'aunque estaciones sucias' 'buena conexión'
 'buena conexión líneas' 'buena conexión líneas horarios'
 'buena conexión líneas horarios nunca' 'conexión líneas'
 'conexión líneas horarios' 'conexión líneas horarios nunca'
 'conexión líneas horarios nunca confiables' 'demasiada gente'
 'demasiada gente servicio' 'demasiada gente servicio malo'
 'eficiente aunque' 'eficiente aunque estaciones'
 'eficiente aunque estaciones sucias' 'estaciones sucias' 'estación metro'
 'estación metro moderna' 'estación metro moderna limpia'
 'estación metro moderna limpia trenes' 'gente servicio'
 'gente servicio malo' 'horarios nunca' 'horarios nunca confiables'
 'limpia trenes' 'limpia trenes lentos' 'líneas horarios'
 'líneas horarios nunca' 'líneas horarios nunca confiables'
 'madrid rápido' 'madrid rápido eficiente'
 'madrid rápido eficiente aunque'
 'madrid rápido eficiente aunque estaciones' 'metro madrid'
 'metro madrid rápido' 'metro madrid rápido eficien

Una vez obtenidos los n-gramas de las reviews filtradas sin los stopwords lo que vamos a hacer es quedarnos de cada n-grama con el sustantivo y su adjetivo adyacente más cercano. Esto es debido a que en español los adjetivos calificativos suelen ir al lado o muy cerca del sustantivo al que califica. De esta manera podremos sacar todas las características.

Nos descargamos e importamos las librerías necesarias para ello. En este caso, hacemos uso de la librería spaCy de python para poder saber la categoría gramatical de las palabras en español:

In [5]:
nlp = spacy.load("es_core_news_sm")

In [6]:
# Crear el mapa: {sustantivo: {(adjetivo))}}
caracteristicasNGramas = defaultdict(set)

for ngrama in ngramas:
    doc = nlp(ngrama)
    sustantivoActual = None
    for token in doc:
        if token.pos_ == "NOUN":
            sustantivoActual = token.lemma_.lower()
        elif token.pos_ == "ADJ" and sustantivoActual:
            caracteristicasNGramas[sustantivoActual].add(token.lemma_.lower())

print(caracteristicasNGramas)

defaultdict(<class 'set'>, {'estación': {'sucio'}, 'conexión': {'horario', 'confiable', 'línea'}, 'gente': {'malo', 'servicio'}, 'servicio': {'malo'}, 'moderna': {'limpio'}, 'horario': {'confiable'}, 'tren': {'lento'}, 'línea': {'horario', 'confiable'}, 'metro': {'rápido', 'moderno', 'eficiente'}})


Se puede apreciar como para las reviews de ejemplo que hemos utilizado hay incongruencias, es decir, se registra que los trenes son lentos pero que el metro es rápido. Además, la librería interpreta algunas palabras como sustantivos o como adjetivos que no son.

### Librerías con redes neuronales

Con los n-gramas solo puedes ver las palabras que tengan cerca. Esto en la mayoria de casos es lo que buscas y necesitas pero sin embargo en otras ocasiones un adjetivo al final de la oración puede referirse a un sustantivo del principio o un adjetivo cerca de un sustantivo puede referirse a otro sustantivo, y esto con los n-gramas no se puede ver. Para ello, haremos uso de las distinntas librerías de python para realizar un análisis más exahustivo de cada review y poder tener resultados más precisos.

El uso de estas librerías va a ser muy similar al de los n-gramas. Vamos a sacar los distintos adjetivos que esten relacionados con los sustantivos correspondientes. De esta manera se ahorra tiempo ya que podemos hacerlo evitando la repetición de los n-gramas.

#### SpaCy

La primera libreía que vamos a utilizar es la de spaCy, previamente utilizada para saber que clase de palabra era cada una en las reviews. Sin embargo, esta librería no solo es capaz de saber la clase gramatical de las palabras sino que además es capaz de decirte con que palabra esta relacionada, que es justo lo que necesitamos saber para extraer las características de las distintas reviews.

In [7]:
reviews = [
    "La estación de metro es moderna y limpia, pero los trenes son lentos.",
    "A veces hay demasiada gente y el servicio es malo.",
    "El metro de Madrid es rápido y eficiente, aunque algunas estaciones están sucias.",
    "Buena conexión con otras líneas, pero los horarios nunca son confiables.",
    "La estacion es muy fea y está muy mal cuidada",
    "Los horarios son muy utiles"
]

In [8]:
nlp = spacy.load("es_core_news_md")

In [9]:
def collect_forms(adj):
    """
    Gather all adjective variants:
      1) If there are no advmods → base lemma ("limpio")
      2) If there are >=1 advmods → join them all + lemma
         e.g. ["muy","mal"] + "cuidada" → "muy mal cuidada"
      3) Recursively include any conjunct ADJ (dep=="conj")
         only if that conjunct has NO of its own subject
    """
    # 1) get all ADV modifiers in document order
    advs = sorted(
        [c for c in adj.children if c.dep_ == "advmod" and c.pos_ == "ADV"],
        key=lambda c: c.i
    )
    adv_lemmas = [c.lemma_.lower() for c in advs]
    base = adj.lemma_.lower()

    forms = set()
    if adv_lemmas:
        forms.add(" ".join(adv_lemmas + [base]))
    else:
        forms.add(base)

    # 2) propagate any simple conjunctions
    for child in adj.children:
        if child.dep_ == "conj" and child.pos_ == "ADJ":
            # only if that conjunct has no own noun subject
            if not any(c.dep_ == "nsubj" and c.pos_ == "NOUN"
                       for c in child.children):
                forms |= collect_forms(child)

    return forms

def find_noun(adj):
    """
    Given an ADJ token, locate its corresponding NOUN (if any) by:
      – direct modifier: dep_=="amod" → head NOUN
      – predicative: child dep_=="nsubj" → that NOUN
      – coordination: dep_=="conj" → recurse to head ADJ
      – copular chains: dep_ in {"acomp","attr"} on a VERB → its nsubj NOUN
    """
    # a) direct modifier
    if adj.dep_ == "amod" and adj.head.pos_ == "NOUN":
        return adj.head

    # b) predicative adjective
    for child in adj.children:
        if child.dep_ == "nsubj" and child.pos_ == "NOUN":
            return child

    # c) coordinated adjective: inherit from head ADJ
    if adj.dep_ == "conj" and adj.head.pos_ == "ADJ":
        return find_noun(adj.head)

    # d) in case of acomp/attr on a verb
    if adj.dep_ in {"acomp", "attr"} and adj.head.pos_ == "VERB":
        for child in adj.head.children:
            if child.dep_ == "nsubj" and child.pos_ == "NOUN":
                return child

    return None

def extract_aspects(reviews):
    """
    Returns: defaultdict(set) mapping noun → set of adjectival phrases.
    """
    nlp = spacy.load("es_core_news_sm")
    aspects = defaultdict(set)

    for doc in nlp.pipe(reviews, batch_size=20):
        for token in doc:
            # only care about ADJ tokens
            if token.pos_ != "ADJ":
                continue
            noun = find_noun(token)
            if noun:
                aspects[noun.lemma_.lower()] |= collect_forms(token)

    return aspects

In [10]:
caracteristicasSpacy = extract_aspects(reviews)

#### Stanza

La otra librería que vamos a utilizar es stanza que es una librería de python desarrollada por Standford. Esta librería es muy similar a la de spaCy, la cual, también permite saber la clase gramatical a la cual pertenece la palabra y con que palabra en la oración esta relacionada. 

In [11]:
nlp = stanza.Pipeline("es", processors="tokenize,mwt,pos,lemma,ner,depparse")

2025-04-27 13:02:04 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-04-27 13:02:04 INFO: Downloaded file to C:\Users\evano\stanza_resources\resources.json
2025-04-27 13:02:05 INFO: Loading these models for language: es (Spanish):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |
| ner       | conll02           |

2025-04-27 13:02:05 INFO: Using device: cpu
2025-04-27 13:02:05 INFO: Loading: tokenize
2025-04-27 13:02:10 INFO: Loading: mwt
2025-04-27 13:02:10 INFO: Loading: pos
2025-04-27 13:02:13 INFO: Loading: lemma
2025-04-27 13:02:15 INFO: Loading: depparse
2025-04-27 13:02:16 INFO: Loading: ner
2025-04-27 13:02:19 INFO: Done loading processors!


##### Procesar cada review

In [12]:
def processReviewsStanza(reviews):

    caracteristicasStanza = defaultdict(set)
    
    for review in reviews:
        doc = nlp(review)
        for sentence in doc.sentences:
            for word in sentence.words:
                if word.upos == "NOUN":
                    sustantivo = word.lemma.lower()
                    headWord = sentence.words[word.head - 1]
                    if headWord.upos == "ADJ":
                        adjetivo = headWord.lemma.lower()
                        caracteristicasStanza[sustantivo].add(adjetivo)
                elif word.upos == "ADJ":
                    adjetivo = word.lemma.lower()
                    headWord = sentence.words[word.head - 1]
                    if headWord.upos == "NOUN":
                        sustantivo = headWord.lemma.lower()
                        caracteristicasStanza[sustantivo].add(adjetivo)

    return caracteristicasStanza

In [13]:

caracteristicasStanza = processReviewsStanza(reviews)

### Comparación de resultados

Vamos a ver que resultados sacan cada opción previamente explicadas y compararlas entre sí. De esta manera nos quedaremos con el que mejor resultados de para utilizarlo con todas y cada una de las reviews de las distintas estaciones de metro que tenemos.

In [14]:
print("Las caracteristicas de los n-gramas \n")
print(caracteristicasNGramas)
print("\n")
print("Las caracteristicas de spacy \n")
print(caracteristicasSpacy)
print("\n")
print("Las caracteristicas de stanza \n")
print(caracteristicasStanza)

Las caracteristicas de los n-gramas 

defaultdict(<class 'set'>, {'estación': {'sucio'}, 'conexión': {'horario', 'confiable', 'línea'}, 'gente': {'malo', 'servicio'}, 'servicio': {'malo'}, 'moderna': {'limpio'}, 'horario': {'confiable'}, 'tren': {'lento'}, 'línea': {'horario', 'confiable'}, 'metro': {'rápido', 'moderno', 'eficiente'}})


Las caracteristicas de spacy 

defaultdict(<class 'set'>, {'estación': {'sucio', 'moderno', 'limpio'}, 'servicio': {'malo'}, 'metro': {'rápido', 'eficiente'}, 'conexión': {'buen'}, 'horario': {'nunca confiable', 'mucho util'}})


Las caracteristicas de stanza 

defaultdict(<class 'set'>, {'estación': {'sucia', 'moderno'}, 'tren': {'lento'}, 'servicio': {'malo'}, 'metro': {'rápido'}, 'conexión': {'confiable', 'buena'}, 'horario': {'confiable', 'util'}, 'estacion': {'feo'}})


Podemos observar como con los n-gramas parece que se sacan más características que con las librerías. Sin embargo, si lo analizamos más profundamente podemos ver como con los n-gramas se añaden palabras que no son sustantivos o que no son adjetivos o que no tienen ninguna relevancia. Esto demuestra que los resultados con las librerías son más precisas. Sin embargo, las librerías alguna característica como que la estación esta limpia también se la ha saltado. 

Entre las dos librerías usadas vemos como dan resultados exactamente iguales, por lo que nos quedaremos con spacy debido a que podemos ver los hijos de una palabra y la palabra con la que esta relacionada. Esto cuando haya más reviews y más largas da más seguridad en que se van a sacar el máximo de características posibles aunque se pierdan algunas como ya hemos podido ver anteriormente. 

Por todo esto la librería que vamos a usar para extraer todas las psoibles características va a ser spacy.

## Obtención de todas las características de las reviews

Una vez realizada la comparación nos quedaremos con la librería spacy que es la que mejor resultados obtuvo. A continuación realizamos la extracción de las caracteristicas para cada una de las distintas estaciones de metro.

Para ello vamos a realizar otra pasada a las reviews y contando su frecuencia para después ver si son relevantes o no

Función auxiliar para contar la frecuencia:

In [ ]:
def collect_forms(adj):
    """
    Gather all adjective variants:
      1) If there are no advmods → base lemma ("limpio")
      2) If there are >=1 advmods → join them all + lemma
         e.g. ["muy","mal"] + "cuidada" → "muy mal cuidada"
      3) Recursively include any conjunct ADJ (dep=="conj")
         only if that conjunct has NO of its own subject
    """
    # 1) get all ADV modifiers in document order
    advs = sorted(
        [c for c in adj.children if c.dep_ == "advmod" and c.pos_ == "ADV"],
        key=lambda c: c.i
    )
    adv_lemmas = [c.lemma_.lower() for c in advs]
    base = adj.lemma_.lower()

    forms = set()
    if adv_lemmas:
        forms.add(" ".join(adv_lemmas + [base]))
    else:
        forms.add(base)

    # 2) propagate any simple conjunctions
    for child in adj.children:
        if child.dep_ == "conj" and child.pos_ == "ADJ":
            # only if that conjunct has no own noun subject
            if not any(c.dep_ == "nsubj" and c.pos_ == "NOUN"
                       for c in child.children):
                forms |= collect_forms(child)

    return forms

def find_noun(adj):
    """
    Dado un ADJ (o participio), localiza su NOUN.
    """
    # a) modificador directo
    if adj.dep_ == "amod" and adj.head.pos_ == "NOUN":
        return adj.head

    # b) predicativo: atributo de un verbo/AUX copulativo
    if adj.dep_ in {"acomp", "attr"} and adj.head.pos_ in {"VERB", "AUX"}:   # ← CAMBIO 1
        for child in adj.head.children:
            if child.dep_ == "nsubj" and child.pos_ == "NOUN":
                return child

    # c) coordenado
    if adj.dep_ == "conj" and adj.head.pos_ == "ADJ":
        return find_noun(adj.head)

    # d) participio como acl
    if adj.dep_ == "acl" and adj.head.pos_ == "NOUN":
        return adj.head

    # e) predicativo con subj propio
    for child in adj.children:
        if child.dep_ == "nsubj" and child.pos_ == "NOUN":
            return child
    
    # f) adjetivo que es la raíz (p. ej. “Los trenes son LENTOS”)
    if adj.dep_ == "ROOT":
        for child in adj.children:
            if child.dep_ == "nsubj" and child.pos_ == "NOUN":
                return child
    
    up = adj
    while up.dep_ != "ROOT":
        up = up.head
        if up.pos_ == "NOUN":
            return up

    return None


def extract_aspects(reviews):
    """
    Returns: defaultdict(set) mapping noun → set of adjectival phrases.
    """
    nlp = spacy.load("es_core_news_sm")
    aspects = defaultdict(Counter)

    for doc in nlp.pipe(reviews, batch_size=20):
        for token in doc:
            if not (token.pos_ == "ADJ" or "Part" in token.morph.get("VerbForm")):
                continue
            if token.dep_ == "conj" and token.head.pos_ == "ADJ":
                continue
            noun = find_noun(token)
            if noun:
                for form in collect_forms(token):
                    noun_key = unidecode(noun.lemma_.lower())
                    aspects[noun_key][form] += 1

    return aspects

In [60]:
results = extract_aspects(reviews)

🔸  NO NOUN: fea ROOT ADJ → head: fea ROOT


In [62]:
reviews

['La estación de metro es moderna y limpia, pero los trenes son lentos.',
 'A veces hay demasiada gente y el servicio es malo.',
 'El metro de Madrid es rápido y eficiente, aunque algunas estaciones están sucias.',
 'Buena conexión con otras líneas, pero los horarios nunca son confiables.',
 'La estacion es muy fea y está muy mal cuidada',
 'Los horarios son muy utiles']

In [63]:
results

defaultdict(collections.Counter,
            {'estacion': Counter({'moderno': 1, 'limpio': 1, 'sucio': 1}),
             'servicio': Counter({'malo': 1}),
             'metro': Counter({'rápido': 1, 'eficiente': 1}),
             'conexion': Counter({'buen': 1}),
             'horario': Counter({'nunca confiable': 1, 'mucho util': 1})})

In [18]:
# Agrega un adjetivo con su frecuencia al sustantivo correspondiente en el diccionario.
def addAdjective(sustantivo, adjetivo):
    for adj, freq in caracteristicas[sustantivo]:
        if adj == adjetivo:
            caracteristicas[sustantivo].remove((adj, freq))
            caracteristicas[sustantivo].add((adj, freq + 1))
            return
    caracteristicas[sustantivo].add((adjetivo, 1))

In [24]:
nlp = spacy.load("es_core_news_sm")
caracteristicas = defaultdict(set)

# Procesar cada review
for review in reviews:
    doc = nlp(review)
    for token in doc:
        if token.pos_ == "ADJ":
            adjetivo = token.lemma_.lower()
            for child in token.children:
                if child.pos_ == "NOUN":
                    sustantivo = child.lemma_.lower()
                    addAdjective(sustantivo, adjetivo)
            if token.head.pos_ == "NOUN":
                sustantivo = token.head.lemma_.lower()
                addAdjective(sustantivo, adjetivo)
        elif token.pos_ == "NOUN":
            sustantivo = token.lemma_.lower()
            if token.head.pos_== "ADJ":
                adjetivo = token.head.lemma_
                addAdjective(sustantivo, adjetivo)

print(caracteristicas) 

defaultdict(<class 'set'>, {'estación': {('sucio', 2), ('moderno', 2)}, 'tren': {('lento', 2)}, 'servicio': {('malo', 2)}, 'metro': {('rápido', 2)}, 'conexión': {('confiable', 1), ('buen', 1)}, 'horario': {('confiable', 2)}})


Como puede haber discrepancias entre las caracteristicas que hemos sacado debido a la variabilidad de todas las reviews solo se van a quedar los que no tengan antonimos para un mismo sustantivo o el antonimo que mayor frecuencia tenga. Por ejemplo, si tenemos que el metro esta limpio con una frecuencia de 7 y que el metro esta sucio con una frecuencia de 5, se quedaría que el metro esta limpio.

Sin embargo, para poder sacar los antonimos de la mejor manera posible vamos a traducirlos al ingles y luego de vuelta al español debido a que en ingles esta mucho mejor optimizado y falla mucho menos que en español.

In [61]:
# Traducir palabra al ingles
def traducir(palabra, sr, tg):
    traduccion = GoogleTranslator(source=sr, target=tg).translate(palabra)
    return traduccion

In [63]:
# Obtener todos los antonimos
def getAntonimos(palabra):
    antonimos = set()
    for sentido in wn.synsets(palabra, pos=wn.ADJ):  # todos los significados de la palabra como adjetivo
        for lema in sentido.lemmas(): # cada sinónimo dentro de ese significado
            for antonimo in lema.antonyms():  # para cada sinónimo, se busca sus antónimos
                antonimos.add(antonimo.name()) # se añade el nombre del antónimo al set
    return antonimos

In [67]:
# Quitar los antonimos que tengan menor frecuencia
def quitarAntonimos(diccionario):
    for sustantivo, adjs in diccionario.items():
        lista = list(adjs) # los pasamos a lista para poder iterar
        nuevosAdjs = set() # adjetivos que van a quedar después del filtro
        procesados = set() # para no comparar un adjetivo más de una vez

        for i, (adj1, freq1) in enumerate(lista):
            if adj1 in procesados:
                continue # si fue comparado se salta

            antonimosIngles = getAntonimos(traducir(adj1,'es','en')) # traducimos las palabras para sacar mejor los antonimos
            antonimos = [traducir(ant, 'en', 'es').lower() for ant in antonimosIngles]
            emparejado = False

            for j in range(i + 1, len(lista)):
                adj2, freq2 = lista[j]
                if adj2 in procesados: # si fue comparado se salta
                    continue
                    
                if adj2 in antonimos: # se guarda el de mayor frecuencia
                    if freq1 > freq2: 
                        nuevosAdjs.add((adj1, freq1))
                    elif freq1 < freq2:
                        nuevosAdjs.add((adj2, freq2))
                        
                    procesados.update([adj1, adj2])
                    emparejado = True
                    break

            if not emparejado: # si no tenía ningún antónimo en la lista se agrega tal cual
                nuevosAdjs.add((adj1, freq1))
                procesados.add(adj1)

        diccionario[sustantivo] = nuevosAdjs     

In [68]:
aux = {'estación': {('sucio', 2), ('moderno', 2)}, 'tren': {('lento', 2)}, 'servicio': {('malo', 2)}, 'conexión': {('confiable', 1), ('buen', 1)}, 'horario': {('confiable', 2)}, "metro": {("limpio", 7), ("rápido", 4), ("lento", 2), ("sucio",5)}}

quitarAntonimos(aux)
print(aux)

{'estación': {('sucio', 2), ('moderno', 2)}, 'tren': {('lento', 2)}, 'servicio': {('malo', 2)}, 'conexión': {('confiable', 1), ('buen', 1)}, 'horario': {('confiable', 2)}, 'metro': {('limpio', 7), ('rápido', 4)}}


Una vez hemos quitado los antonimos con menos frecuencia, es decir, caracteristicas irrelevantes, vamos a añadir y guardar las características sin la frecuencia.

In [66]:
caracteristicasFinal = {}

def quitarFrecuencias(dicF, dic):
    for sustantivo, adjetivos in dicF.items():
        dic[sustantivo] = {adj for adj, _ in adjetivos}
    return dic

caracteristicasFinal = quitarFrecuencias(aux, caracteristicasFinal)
print(caracteristicasFinal)

{'estación': {'sucio', 'moderno'}, 'tren': {'lento'}, 'servicio': {'malo'}, 'metro': {'limpio', 'rápido'}, 'conexión': {'buen', 'confiable'}, 'horario': {'confiable'}}
